In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 300
start_epoch = 0
train_batch = 110
test_batch = 110
lr = 0.04
schedule = [75,150,225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b1/siamese' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
# iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# constrastive
thresh = 0.5

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

# Dataset

In [9]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch)
val_ = SiameseNetworkDataset(datasets.ImageFolder(val_dir), transform=val_aug, should_invert=False)
val_loader = DataLoader(val_, shuffle=True, num_workers=num_workers, batch_size=test_batch)

# Model

In [10]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    pt = torch.load(pretrained)['state_dict']
    model.load_state_dict(pt)

In [11]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.17M


# Loss

In [12]:
criterion = ContrastiveLoss(margin=1.0).cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

In [14]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Train

In [15]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            target_index = targets[targets==0]
            target_index = target_index.long().cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs0.size(), lam)
            inputs0[target_index, :, bbx1:bbx2, bby1:bby2] = inputs1[target_index, :, bbx1:bbx2, bby1:bby2]
        

        
        outputs0 = model(inputs0)
        outputs1 = model(inputs1)
        
        loss = criterion(outputs0, outputs1, targets)
            
        # compute output
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

        # measure accuracy and record loss
        pred = outputs.data
        pred[pred < thresh] = 0.
        pred[pred >= thresh] = 1.
        prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
        
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [16]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            loss = criterion(outputs0, outputs1, targets)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            # measure accuracy and record loss
            pred = outputs.data
            pred[pred < thresh] = 0.
            pred[pred >= thresh] = 1.
            prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # plot progress
            bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                        batch=batch_idx + 1,
                        size=len(val_loader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        total=bar.elapsed_td,
                        eta=bar.eta_td,
                        loss=losses.avg,
                        top1=top1.avg,)
            bar.next()
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
             batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
        bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)

    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 300] LR: 0.040000
1168/1168 Data:0.011 | Batch:0.574 | Total:0:12:03 | ETA:0:00:01 | Loss:0.27401352973196785 | top1:0.49848095349380694
292/292 Data:0.027 | Batch:0.802 | Total:0:01:01 | ETA:0:00:00 | Loss:0.27290395770767395 | top1:0.4988785046728972

Epoch: [2 | 300] LR: 0.042400
1168/1168 Data:0.015 | Batch:0.597 | Total:0:13:49 | ETA:0:00:01 | Loss:0.2611437612041614 | top1:0.5002181194983252
292/292 Data:0.046 | Batch:0.275 | Total:0:01:16 | ETA:0:00:00 | Loss:0.26491390395665837 | top1:0.5011214953271028

Epoch: [3 | 300] LR: 0.044800
1168/1168 Data:0.023 | Batch:0.658 | Total:0:14:55 | ETA:0:00:01 | Loss:0.25914395962552117 | top1:0.4993612214691906
292/292 Data:0.018 | Batch:0.250 | Total:0:01:03 | ETA:0:00:00 | Loss:0.25919572001677066 | top1:0.5030218068535826

Epoch: [4 | 300] LR: 0.047200
1168/1168 Data:0.047 | Batch:0.688 | Total:0:13:16 | ETA:0:00:01 | Loss:0.257877073613823 | top1:0.4997351406091766
292/292 Data:0.028 | Batch:0.189 | Total:0:01:01 | ETA:0:0

1168/1168 Data:0.009 | Batch:0.541 | Total:0:11:25 | ETA:0:00:01 | Loss:0.20271335913773367 | top1:0.6873101191867259
292/292 Data:0.009 | Batch:0.153 | Total:0:00:47 | ETA:0:00:00 | Loss:0.0929545858196009 | top1:0.9891277258566978

Epoch: [33 | 300] LR: 0.116800
1168/1168 Data:0.009 | Batch:0.603 | Total:0:11:23 | ETA:0:00:01 | Loss:0.20192106747433172 | top1:0.6870997896704838
292/292 Data:0.007 | Batch:0.148 | Total:0:00:51 | ETA:0:00:00 | Loss:0.0555863542638928 | top1:0.9901557632398754

Epoch: [34 | 300] LR: 0.119200
1168/1168 Data:0.009 | Batch:0.593 | Total:0:11:23 | ETA:0:00:01 | Loss:0.20162976402294563 | top1:0.6882371270546078
292/292 Data:0.009 | Batch:0.154 | Total:0:00:51 | ETA:0:00:00 | Loss:0.08490532774745119 | top1:0.9872274143302181

Epoch: [35 | 300] LR: 0.121600
1168/1168 Data:0.008 | Batch:0.593 | Total:0:11:24 | ETA:0:00:01 | Loss:0.20141425612602054 | top1:0.6894523642595622
292/292 Data:0.008 | Batch:0.151 | Total:0:00:52 | ETA:0:00:00 | Loss:0.07748952263797

1168/1168 Data:0.008 | Batch:0.596 | Total:0:11:18 | ETA:0:00:01 | Loss:0.17715319041114846 | top1:0.7263379294227623
292/292 Data:0.009 | Batch:0.146 | Total:0:00:51 | ETA:0:00:00 | Loss:0.07605526787446482 | top1:0.9076323987538941

Epoch: [64 | 300] LR: 0.159369
1168/1168 Data:0.009 | Batch:0.573 | Total:0:11:17 | ETA:0:00:01 | Loss:0.17619062042711325 | top1:0.7288696735997507
292/292 Data:0.009 | Batch:0.156 | Total:0:00:51 | ETA:0:00:00 | Loss:0.06303253152010226 | top1:0.9350778816199377

Epoch: [65 | 300] LR: 0.159260
1168/1168 Data:0.009 | Batch:0.575 | Total:0:11:18 | ETA:0:00:01 | Loss:0.17431357956228852 | top1:0.7304821998909402
292/292 Data:0.008 | Batch:0.153 | Total:0:00:51 | ETA:0:00:00 | Loss:0.06576938046449998 | top1:0.9383489096573209

Epoch: [66 | 300] LR: 0.159142
1168/1168 Data:0.009 | Batch:0.599 | Total:0:11:19 | ETA:0:00:01 | Loss:0.17056537926477966 | top1:0.7398145984264236
292/292 Data:0.009 | Batch:0.142 | Total:0:00:52 | ETA:0:00:00 | Loss:0.056219737130

1168/1168 Data:0.013 | Batch:0.652 | Total:0:12:19 | ETA:0:00:01 | Loss:0.1292696843544642 | top1:0.8162888525356392
292/292 Data:0.014 | Batch:0.146 | Total:0:00:55 | ETA:0:00:00 | Loss:0.06519606962359026 | top1:0.9158566978193147

Epoch: [95 | 300] LR: 0.015203
1168/1168 Data:0.014 | Batch:0.603 | Total:0:11:54 | ETA:0:00:01 | Loss:0.12876989210428288 | top1:0.8160084131806496
292/292 Data:0.045 | Batch:0.324 | Total:0:00:55 | ETA:0:00:00 | Loss:0.060549149046596897 | top1:0.9195327102803739

Epoch: [96 | 300] LR: 0.015166
1168/1168 Data:0.014 | Batch:0.700 | Total:0:12:18 | ETA:0:00:01 | Loss:0.12891534049999234 | top1:0.8172937602243515
292/292 Data:0.028 | Batch:0.200 | Total:0:00:56 | ETA:0:00:00 | Loss:0.06986832070480625 | top1:0.9079439252336449

Epoch: [97 | 300] LR: 0.015128
1168/1168 Data:0.010 | Batch:0.593 | Total:0:11:59 | ETA:0:00:01 | Loss:0.12726135533414748 | top1:0.8185635273038872
292/292 Data:0.023 | Batch:0.207 | Total:0:00:58 | ETA:0:00:00 | Loss:0.065863757881